Para aplicar o primeiro método, de regressão, vamos primeiro importar o arquivo .csv gerado anteriormente e verificar suas colunas.
Vamos então adicionar valores padrões para as colunas de Publisher e Year, já que por algum motivo, o DataSet trouxe alguns valores destas colunas como nulos.
Para a coluna Publisher, é difícil termos uma métrica já que ela é uma coluna anteriormente transformada de texto para número, então optamos por atribuir o valor padrão 0, que apesar de não ser realista, não deve ter um grande impacto nas predições já que os campus nulos são uma minúscula minoria no DataSet.
Já para a coluna Year, como ela já se tratava de um campo número, decidimos calcular a média de todos os anos apresentados no DataSet (sem peso), chegando assim no valor de 2000.

Então, removemos a coluna Rank, já que ela não seria uma boa coluna para ser contabilizada na predição.
Por fim, dvidimos os valores das colunas que pretendemos usar como colunas preditivas e coluna target para variáveis x e y respectivamente, onde a variável x é a variável preditiva e a y é a variável target. E então exibimos os 5 primeiros valores do documento csv para checar se está tudo em ordem.

In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Reading the .csv file used for the tests.
from google.colab import files
uploaded = files.upload()
df = pd.read_csv('DataSet Games.csv')


# Checking which columns this csv file has, and formatting the Publisher and Year with default values
# whenever it has NaN values.
# Note that for the Publisher, we can't know which value it should have so it is given 0
# (we should also not take the mean since originally, this column was a text one, so it may not be accurate),
# but on Year we can make the mean value of all years present on the DataSet, which gives us "2000" .
df.columns
df['Publisher'] = df['Publisher'].fillna(0)
df['Year'] = df['Year'].fillna(2000)

# Drop any uneeded column (Rank can still be used as predictive variable, but it's not suitable for it).
df.drop('Rank',axis=1,inplace=True)

# Assigning the predictive and target values for X and Y respectively and showing a preview of the data to
# check if it is correct.
X = df[['Platform', 'Year', 'Genre', 'Publisher']]
y = df['Global_Sales']
df.head()




Saving DataSet Games.csv to DataSet Games.csv


,Platform,Year,Genre,Publisher,Global_Sales
0,0.70,2006.0,0.57,2.54,82.74
1,2.56,1985.0,0.94,2.54,40.24
2,0.70,2008.0,0.59,2.54,35.82
3,0.70,2009.0,0.57,2.54,33.00
4,2.61,1996.0,0.62,2.54,31.37


Agora, dividimos os nossos dados de x e y nas variáveis de teste e treino. Nota-se que optamos por remover o random_state para podermos ter novos valores escolhidos aleatoriamente a cada uso da aplicação, o que nos permite ter uma maior diversidade de acertividade.
Ainda, como o nosso dataset é composto por pouquíssimas variáveis preditoras, escolhemos alocar uma quantidade menor de valores para teste, pois desta forma temos mais valores para serem usados no treino, o que em teoria aumentaria a precisão do algorítmo.
Depois disso, é criado o modelo de regressão linear e atribuido as variáveis de treino a ele, as quais serão usadas para faze-lo "aprender".
Após isso, o algorítmo receberá os valores previstos e exibiremos os 5 primeiros valores do teste para, novamente, checar se tudo foi realizado como planejado.

In [50]:
# Adding the train and test variables accordingly with how much and which data we want to
# keep as test and train.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

from sklearn.linear_model import LinearRegression

# Creating predictor model.
modelopreditor = LinearRegression()


# Adding the train variables to the model so it can be used to learn.
modelopreditor.fit(X_train,y_train)

# Predicting the test values based on what was learned before.
predictions = modelopreditor.predict(X_test)


# printing the predictor variables so we can check which set of data were picked.
X_test.head()


,Platform,Year,Genre,Publisher
8682,0.61,1996.0,0.59,0.52
12977,0.38,2010.0,0.42,0.20
15305,0.39,2005.0,0.47,0.00
11997,0.78,2011.0,0.53,0.53
10900,0.83,2015.0,0.53,0.53


Exibimos então os valores previstos para Global_Sales com base no que foi aprendido anteriormente. Esta linha ainda serve apenás para fins de exibição.

In [51]:

# Showing the predicted values for Global_Sales for that same data.
predictions

array([ 0.61131215,  0.07521259, -0.06670126, ...,  0.78880127,
        0.9474914 ,  0.36379441])

O mesmo para esta linha de código, mas agora estamos exibindo os valores reas de y, os quais serão utilizados para "corrigir" o que foi previsto pelo algoritmo.

In [52]:

# Showing the real values for Global_Sales for that same data.
y_test

8682     0.16
12977    0.05
15305    0.02
11997    0.07
10900    0.09
         ... 
3535     0.57
13169    0.05
8937     0.15
4385     0.45
14937    0.02
Name: Global_Sales, Length: 1658, dtype: float64

Por fim, exibimos as métricas da margem de erro que o algoritmo obteve entre o valor predito e o valor real, destacando com o método MSE, as falhas. Desta forma podemos ter uma ideia do desempenho do algoritmo.

In [53]:
# Importing metrics library and print the means so we can have a basic analysis
# of how much error the algorithm had on its predicition.
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))


MAE: 0.4868951464859218
MSE: 1.5481313914623618
RMSE: 1.244239282237288


Ao final, vemos então que a margem de erro do algoritmo foi considerável. Isto se deve ao fato dele não possuir variáveis preditoras suficientes para poder treinar uma resposta mais precisa. Alterar a quantidade de dados para teste, e a aleatoriedade dos dados afetam, e podem ajudar bastante na definição destes valores, mas certamente não os mitigam.